In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
## Let's start by loading our data

# Starting with the labels
with open("./data/processed_training_labels.csv") as labels_file:
    labels_string = labels_file.read()
    labels = np.array(labels_string.split(','), dtype=int)
    
# Recall we had 60000 images. Let's make sure we didn't lose anythin
assert len(labels) == 60000

# Now for the images
images = []
with open("./data/processed_training_images") as images_file:
    raw_image_strings = images_file.readlines()
    for img_string in raw_image_strings:
        img_flat = np.array(img_string.split(","), dtype=np.double)
        img = np.reshape(img_flat, (28,28))
        images.append(img)
        
# Again, let's do some random spot checking to make sure everything is as we expect
assert len(images) == 60000
i1,i2,i3 = np.random.randint(0, 60000, 3)
assert images[i1].shape == (28,28)
assert images[i2].shape == (28,28)
assert images[i3].shape == (28,28)

In [ ]:
# And again, we'll just print out some images and their labels for good measure 

fig = plt.figure(figsize=(10,10))
fig.add_subplot(1,4,1)
plt.text(9,40,f"label: {labels[i1]}")
plt.imshow(images[i1], cmap='gray')
fig.add_subplot(1,4,2)
plt.text(10,40,f"label: {labels[i2]}")
plt.imshow(images[i2], cmap='gray')
fig.add_subplot(1,4,3)
plt.text(11,40,f"label: {labels[i3]}")
plt.imshow(images[i3], cmap='gray')

In [3]:
## Okay! Now the fun begins.
# To start, let's just get everything over to Torch
t_labels = torch.tensor(labels)
t_images = torch.tensor(images)

train_images = t_images.reshape(60000, 1, 784).float()
train_labels = torch.zeros(60000, 1, 10)
for i, label in enumerate(t_labels):
    train_labels[i][0][label.item()] = 1

#train_labels = train_labels.flip(0)
#train_images = train_images.flip(0)

In [ ]:
# And we can take a look at their shapes
import matplotlib.pyplot as plt
z = np.random.randint(0, 60000)
plt.imshow(train_images[z][0].reshape(28,28), cmap='gray')
print(train_labels[z][0])

In [125]:
# TODO: !DF! Try just one fully connected layer
# TODO: !DF! Write explanation on MSELoss()
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1,1,3)
        #self.drop1 = nn.(389,p=.2) 
        self.fc1 = nn.Linear(782,100) # 80
        self.drop2 = nn.Dropout()
        self.fc2 = nn.Linear(100, 10)
        #self.fc3 = nn.Linear(100, 10) # 80
        
    def forward(self, x):
        #x = F.max_pool1d(F.sigmoid(self.conv1(x)), (2))  ## TODO: Test this again against whatever you leave here
        x = F.sigmoid(self.conv1(x))
        #x = self.drop1(x)
        x = F.sigmoid(self.fc1(x))
        #x = F.sigmoid(self.fc2(x))
        #x = self.drop2(x)
        #x = F.relu(self.fc2(x))
        return self.fc2(x)
        #return self.fc3(x)

## 100 -> ~76 ~78 ~78 ~78%
## 80 -> ~78 ~76 ~ 77 ~78%

## 100 with max_pool -> 71%, 73% 71%

In [126]:
net = Net()
net = net
# https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
# TODO:!DF! test with and without these weights
net.fc1.weight.data.uniform_(-1.0/np.sqrt(782),1.0/np.sqrt(782))
_ = net.fc2.weight.data.uniform_(-1.0/np.sqrt(100),1.0/np.sqrt(100))
#net.fc2.bias.data
#net.fc3.weight.data.uniform_(-1.0/np.sqrt(80),1.0/np.sqrt(80))


In [127]:
criterion = nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01)
for epoch in range(8):
    running_loss = 0.0
    for i in range(0,len(train_images), 4):
        net.zero_grad()
        out = net(train_images[i:i+4])
        loss = criterion(out, train_labels[i:i+4])
        loss.backward()
        running_loss += loss.item()
        if i % 2000 == 0:
            ## I recommend doing things like this. It helped me catch mistakes
            #print(batch[0][0])
            #plt.imshow(batch[0][0].reshape((28,28)), cmap='gray')
            #plt.show()
            #print(targets[0])
            print(f"Epoch: {epoch}, Batch: {i}, running_loss: {running_loss}")
            running_loss = 0.0
        for f in net.parameters():
            f.data.sub_(f.grad.data * 0.01)
        
    

Epoch: 0, Batch: 0, running_loss: 0.3842152953147888
Epoch: 0, Batch: 2000, running_loss: 46.38825993984938
Epoch: 0, Batch: 4000, running_loss: 43.24665227532387
Epoch: 0, Batch: 6000, running_loss: 41.554694175720215
Epoch: 0, Batch: 8000, running_loss: 40.372930347919464
Epoch: 0, Batch: 10000, running_loss: 38.94594490155578
Epoch: 0, Batch: 12000, running_loss: 37.53005349636078
Epoch: 0, Batch: 14000, running_loss: 36.75769066810608
Epoch: 0, Batch: 16000, running_loss: 35.99681656435132
Epoch: 0, Batch: 18000, running_loss: 34.7306363992393
Epoch: 0, Batch: 20000, running_loss: 33.106588777154684
Epoch: 0, Batch: 22000, running_loss: 32.05949483066797
Epoch: 0, Batch: 24000, running_loss: 31.757390279322863
Epoch: 0, Batch: 26000, running_loss: 30.47219653427601
Epoch: 0, Batch: 28000, running_loss: 30.163912538439035
Epoch: 0, Batch: 30000, running_loss: 29.34362237341702
Epoch: 0, Batch: 32000, running_loss: 29.73763306811452
Epoch: 0, Batch: 34000, running_loss: 28.5176196359

Epoch: 4, Batch: 52000, running_loss: 18.332820639014244
Epoch: 4, Batch: 54000, running_loss: 18.623434809036553
Epoch: 4, Batch: 56000, running_loss: 17.78602289687842
Epoch: 4, Batch: 58000, running_loss: 17.89673050865531
Epoch: 5, Batch: 0, running_loss: 0.030313873663544655
Epoch: 5, Batch: 2000, running_loss: 18.981735589914024
Epoch: 5, Batch: 4000, running_loss: 17.458728382829577
Epoch: 5, Batch: 6000, running_loss: 18.296514101792127
Epoch: 5, Batch: 8000, running_loss: 18.69200872629881
Epoch: 5, Batch: 10000, running_loss: 18.863675351720303
Epoch: 5, Batch: 12000, running_loss: 18.441970754880458
Epoch: 5, Batch: 14000, running_loss: 19.432088705711067
Epoch: 5, Batch: 16000, running_loss: 20.328678723424673
Epoch: 5, Batch: 18000, running_loss: 19.362796218367293
Epoch: 5, Batch: 20000, running_loss: 17.50675371987745
Epoch: 5, Batch: 22000, running_loss: 18.013050688896328
Epoch: 5, Batch: 24000, running_loss: 19.339076307602227
Epoch: 5, Batch: 26000, running_loss: 18.

In [ ]:
plt.imshow(t_images[750], cmap='gray')

In [ ]:
torch.argmax(net(t_images[750].flatten().float().unsqueeze(0).unsqueeze(0)))

In [119]:
with open("./data/processed_testing_labels.csv") as labels_file:
    labels_string = labels_file.read()
    testing_labels = np.array(labels_string.split(','), dtype=int)
    
# Recall we had 60000 images. Let's make sure we didn't lose anythin
assert len(testing_labels) == 10000

# Now for the images
testing_images = []
with open("./data/processed_testing_images") as images_file:
    raw_image_strings = images_file.readlines()
    for img_string in raw_image_strings:
        img_flat = np.array(img_string.split(","), dtype=np.double)
        img = np.reshape(img_flat, (28,28))
        testing_images.append(img)

In [9]:
t_test_labels = torch.tensor(testing_labels)
t_test_images = torch.tensor(testing_images)

In [128]:
test_imgs = t_test_images
test_labels = t_test_labels
correct = []
for i,img in enumerate(test_imgs):
    res = torch.argmax(net(img.flatten().float().unsqueeze(0).unsqueeze(0)))
    targ = test_labels[i]
    if res == targ:
        correct.append(1)
    else:
        correct.append(0)

print(f"{sum(correct)} / {len(correct)}")

8766 / 10000


In [ ]:
img = t_test_images[9496]
plt.imshow(img, cmap='gray')

In [ ]:
torch.argmax(net(img.flatten().float().unsqueeze(0).unsqueeze(0)))

In [ ]:
img.shape

In [ ]:
img.flatten().shape

In [ ]:
##torch.save(net, "./models/1dC2fc85")
##torch.save(net.state_dict(), "./models/1dC2fc85.state_dict")

In [ ]:
train_labels.flip(0)

In [ ]:
my_img = train_images[0][0]
my_conv1d = nn.Conv1d(1,2,2)
my_conv1d(my_img.unsqueeze(0).unsqueeze(0))[0]


In [ ]:
f = nn.Conv1d(1,1,3)

In [ ]:
f(train_images[0].unsqueeze(0)).shape

In [ ]:
104/26